In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# بستن خوشه‌های Dask قبلی
from dask.distributed import Client
try:
    client = Client()
    client.close()
except:
    pass

# آزاد کردن حافظه RAM
import gc
gc.collect()

# بررسی حافظه آزاد
!free -m

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:40863
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40309'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43269'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:46293 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:46293
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:50192
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:43237 name: 1
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:43237
INFO:distributed.core:Starting established connection to tcp://127

               total        used        free      shared  buff/cache   available
Mem:           12977        1027        8951           1        2998       11668
Swap:              0           0           0


In [ ]:
# parallel_pipeline_dask_google_drive_updated.py
# -------------------------------------
# نسخه‌ی موازی با Dask برای پیش‌پردازش و ذخیره در Google Drive
# مناسب اجرا در Google Colab، و آماده برای time-series forecasting با نگاه ۶۰ گام

# 1) نصب و وارد کردن کتابخانه‌ها
!pip install --quiet dask[complete] pyarrow fsspec h5py
from google.colab import drive
from dask.distributed import Client
import dask.dataframe as dd
import numpy as np
import os
import dask
import gc

# 2) اتصال Google Drive
# drive.mount('/content/drive')  # هنگام اجرا در Colab فعال شود


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.3 MB/s eta 0:00:00


In [ ]:

# 3) تنظیمات اولیه
DATA_DIR = '/content/drive/MyDrive/synthetic_fdms_reduced_F'
LOOKBACK_STEPS = 60
FEATURE_COLUMNS = [
    'Viscosity_cP', 'Temperature_C', 'Shear_Rate', 'Salinity_ppm', 'pH_Level'
]
LABEL_COLUMN = 'emulsion_risk'
CLEAN_PARQUET = '/content/drive/MyDrive/cleaned_scaled.parquet'



In [ ]:
# 4) آزادسازی حافظه و بستن کلاینت‌های قبلی
try:
    client = Client()
    client.close()
except:
    pass
gc.collect()




INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:39681
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35205'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45973'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:34555 name: 1
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:34555
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:41632
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:39069 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:39069
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:41646
INFO:distributed.scheduler:Receive client connection: Client-15848cbf-64b4-11f0-821e-0242ac1c000c
INFO:distributed.core:Starting establish

448

In [ ]:
# 5) راه‌اندازی Dask با تنظیمات بهینه
client = Client(n_workers=1, threads_per_worker=4, memory_limit='6GB')
print(client)

dask.config.set({
    'distributed.scheduler.work-stealing': False,
    'distributed.worker.memory.spill': 0.6
})



/usr/local/lib/python3.11/dist-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46241 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:33701
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:46241/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44817'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:40415 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40415
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:39550
INFO:distributed.scheduler:Receive client connection: Client-34dc4678-64b4-11f0-821e-0242ac1c000c
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:39564


<Client: 'tcp://127.0.0.1:33701' processes=1 threads=4, memory=5.59 GiB>


/usr/local/lib/python3.11/dist-packages/distributed/client.py:1582: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | None   | None      | 4.4.4   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [ ]:
# 6) خواندن فایل‌های Parquet و پارتیشن‌بندی
# فقط ستون‌های مورد نیاز به علاوه timestamp را بارگذاری می‌کنیم
columns_to_load = FEATURE_COLUMNS + ['timestamp']
ddf = dd.read_parquet(f"{DATA_DIR}/*.parquet", engine='pyarrow', columns=columns_to_load)
ddf = ddf.repartition(npartitions=150).persist()
gc.collect()



41

In [ ]:
# 7) تبدیل timestamp و مرتب‌سازی کلی درون پارتیشن
from dask.dataframe import to_datetime

ddf['timestamp'] = to_datetime(ddf['timestamp'])
ddf = ddf.map_partitions(lambda df: df.sort_values('timestamp')).persist()
gc.collect()




521

In [ ]:
# 8) پر کردن مقادیر گمشده

def fill_missing(df):
    return df.interpolate().fillna(method='bfill').fillna(method='ffill')

# فقط روی FEATURE_COLUMNS اعمال می‌شود
ddf[FEATURE_COLUMNS] = ddf[FEATURE_COLUMNS].map_partitions(
    lambda df: fill_missing(df),
    meta={col: 'float64' for col in FEATURE_COLUMNS}
).persist()
gc.collect()



1105

In [ ]:
# 9) برچسب‌گذاری emulsion_risk
def compute_label(df):
    crit = (
        (df['Viscosity_cP'] > 80).astype(int) +
        (df['Salinity_ppm'] > 100000).astype(int) +
        ((df['pH_Level'] < 5) | (df['pH_Level'] > 9)).astype(int) +
        ((df['Shear_Rate'] < 50) | (df['Shear_Rate'] > 1000)).astype(int) +
        (df['Temperature_C'] > 90).astype(int)
    )
    dv = df['Viscosity_cP'].diff().abs().fillna(0)
    dt = df['Temperature_C'].diff().abs().fillna(0)
    delta = ((dv > 15).astype(int) + (dt > 10).astype(int))
    df[LABEL_COLUMN] = ((crit + delta) >= 3).astype(int)
    return df

# اعمال برچسب
ddf = ddf.map_partitions(
    compute_label,
    meta={**{col: 'float64' for col in FEATURE_COLUMNS}, 'timestamp': 'datetime64[ns]', LABEL_COLUMN: 'int64'}
).persist()
gc.collect()



41

In [ ]:
# 10) نرمال‌سازی: محاسبه mins و maxs با دقیق گرفتن فقط FEATURE_COLUMNS
ddf_features = ddf[FEATURE_COLUMNS]
mins = ddf_features.min().compute()
maxs = ddf_features.max().compute()

for col in FEATURE_COLUMNS:
    ddf[f'scaled_{col}'] = (ddf[col] - mins[col]) / (maxs[col] - mins[col])



In [ ]:
# 11) ذخیره در Google Drive (Parquet)
# فقط ستون‌های نرمال‌شده، برچسب و timestamp ذخیره می‌شوند
final_columns = [f'scaled_{col}' for col in FEATURE_COLUMNS] + [LABEL_COLUMN, 'timestamp']

if os.path.exists(CLEAN_PARQUET):
    os.remove(CLEAN_PARQUET)
ddf[final_columns].to_parquet(CLEAN_PARQUET, engine='pyarrow')
print("✅ Preprocessed & scaled data saved to:", CLEAN_PARQUET)



✅ Preprocessed & scaled data saved to: /content/drive/MyDrive/cleaned_scaled.parquet


In [ ]:
# 12) بستن کلاینت داسک
client.close()

INFO:distributed.scheduler:Remove client Client-34dc4678-64b4-11f0-821e-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:39564; closing.
INFO:distributed.scheduler:Remove client Client-34dc4678-64b4-11f0-821e-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-34dc4678-64b4-11f0-821e-0242ac1c000c
INFO:distributed.scheduler:Retire worker addresses (stimulus_id='retire-workers-1752942340.5948477') (0,)
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:44817'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:39550; closing.
INFO:distributed.scheduler:Remove worker addr: tcp://127.0.0.1:40415 name: 0 (stimulus_id='handle-worker-cleanup-1752942341.4902208')
INFO:distributed.scheduler:Lost all workers
INFO:distributed.nanny:Nanny at 'tcp://127.0.0.1:44817' closed.
INFO:distributed.scheduler:Closing scheduler. Reason: unknown


In [ ]:
temp_ddf = dd.read_parquet(CLEAN_PARQUET, engine='pyarrow', columns=final_columns)
print(temp_ddf.head(10))


     scaled_Viscosity_cP  scaled_Temperature_C  scaled_Shear_Rate  \
3               0.514517              0.695121           0.354446   
89              0.543801              0.524005           0.917251   
173             0.474023              0.452287           0.595321   
272             0.377784              0.704805           0.404524   
295             0.827568              0.538432           0.304173   
326             0.323969              0.803894           0.498727   
577             0.612987              0.566330           0.789184   
599             0.498401              0.475444           0.800334   
751             0.340332              0.557348           0.352630   
794             0.526285              0.447311           0.802420   

     scaled_Salinity_ppm  scaled_pH_Level  emulsion_risk           timestamp  
3               0.339997         0.612494              1 2023-01-01 00:00:03  
89              0.234999         0.417442              0 2023-01-01 00:01:29  
173